# **Programa de Pós-Graduação em Computação - INF/UFRGS**
### Disciplina CMP263 - Aprendizagem de Máquina
#### *Aluno: Edmar Junyor Bevilaqua*
<br>

### Obtendo os dados do moodle (fiz uma cópia no meu Drive para evitar autenticação):
---

In [ ]:
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1CKWPfJlja9Vzk5gTR-eIsTh39Yt7kYeo' -O AtividadePraticaKNN.zip

In [ ]:
# Descompactando o arquivo -> `-q` suprime textos desnecessários.
!unzip -q AtividadePraticaKNN.zip

# Removendo arquivos desnecessários.
!rm -rf AtividadePraticaKNN.zip
!rm -rf __MACOSX/

### Importando bibliotecas:
---

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

### Lendo os arquivos `.txt`
---

In [ ]:
# Dados Originas 2 Features
orig_train_df_2f = pd.read_csv('/content/AtividadePraticaKNN/Dados_Originais_2Features/TrainingData_2F_Original.txt', sep = '\t')
orig_test_df_2f = pd.read_csv('/content/AtividadePraticaKNN/Dados_Originais_2Features/TestingData_2F_Original.txt', sep = '\t')

# Dados Originas 11 Features
orig_train_df_11f = pd.read_csv('/content/AtividadePraticaKNN/Dados_Originais_11Features/TrainingData_11F_Original.txt', sep = '\t')
orig_test_df_11f = pd.read_csv('/content/AtividadePraticaKNN/Dados_Originais_11Features/TestingData_11F_Original.txt', sep = '\t')

# Dados Normalizados 2 Features
norm_train_df_2f = pd.read_csv('/content/AtividadePraticaKNN/Dados_Normalizados_2Features/TrainingData_2F_Norm.txt', sep = '\t')
norm_test_df_2f = pd.read_csv('/content/AtividadePraticaKNN/Dados_Normalizados_2Features/TestingData_2F_Norm.txt', sep = '\t')

# Dados Normalizados 11 Features
norm_train_df_11f = pd.read_csv('/content/AtividadePraticaKNN/Dados_Normalizados_11Features/TrainingData_11F_Norm.txt', sep = '\t')
norm_test_df_11f = pd.read_csv('/content/AtividadePraticaKNN/Dados_Normalizados_11Features/TestingData_11F_Norm.txt', sep = '\t')

### Plot dos dados para entender distribuição:
---
Gerado com auxílio do Gemini (Google)

In [ ]:
# prompt: Create a plot from norm_train_df_2f that show features 'total.sulfur.dioxide' and 'citric.acid' in axis, then color them based on 'class', then insert the data points from norm_test_df_2f and make them black dots, with their respective ID above

import matplotlib.pyplot as plt
plt.figure(figsize=(20, 11))
sns.scatterplot(x='total.sulfur.dioxide', y='citric.acid', hue='class', data=norm_train_df_2f)
plt.scatter(norm_test_df_2f['total.sulfur.dioxide'], norm_test_df_2f['citric.acid'], color='black', label='Test Data', marker='^')

for index, row in norm_train_df_2f.iterrows():
    plt.annotate(index, (row['total.sulfur.dioxide']-0.003, row['citric.acid']+0.01), size=8)

for index, row in norm_test_df_2f.iterrows():
    plt.annotate(row['ID'], (row['total.sulfur.dioxide'], row['citric.acid']+0.01))

plt.xlabel('total.sulfur.dioxide')
plt.ylabel('citric.acid')
plt.title('Scatter Plot of total.sulfur.dioxide vs citric.acid')
plt.legend()
plt.show()


In [ ]:
def drop_id_column(df):
  df.drop('ID', axis = 1, inplace = True)

## Criando função para separar vetor de atributos (X) e alvos (y):
---

In [ ]:
def split_and_train(df, k):
  X = df.copy()
  y = X.pop('class')
  drop_id_column(X)
  knn = KNeighborsClassifier(n_neighbors = k)
  knn.fit(X, y)
  return knn

In [ ]:
def create_plots_and_neighbors(train_df, test_df, k_list:list):
    X_test = test_df.copy()
    drop_id_column(X_test)
    y_test = X_test.pop('class')

    for k in k_list:
        knn = split_and_train(train_df, k)

        print(f"Plot para K = {k}")
        plt.figure(figsize=(20, 11))

        # -------

        # Plotar os dados de treinamento
        sns.scatterplot(x='total.sulfur.dioxide', y='citric.acid', hue='class', data=train_df)

        # Plotar os dados de teste
        plt.scatter(test_df['total.sulfur.dioxide'], test_df['citric.acid'], color='black', label='Test Data', marker='^')

        # Anotar os IDs dos dados de teste
        for index, row in test_df.iterrows():
            plt.annotate(row['ID'], (row['total.sulfur.dioxide'], row['citric.acid'] + 0.01))

        # Anotar os índices dos dados de treinamento
        for index, row in train_df.iterrows():
            plt.annotate(row['ID'], (row['total.sulfur.dioxide'] - 0.003, row['citric.acid'] + 0.01), size=8)

        # Adicionar linhas pontilhadas para os vizinhos mais próximos
        for test_index, test_row in test_df.iterrows():
            # Encontrar os vizinhos mais próximos para o ponto de teste atual
            distances, indices = knn.kneighbors(test_df.loc[[test_index], ['total.sulfur.dioxide', 'citric.acid']])

            # Iterar sobre os vizinhos e desenhar linhas pontilhadas
            for neighbor_index in indices[0]:
                neighbor_row = train_df.iloc[neighbor_index]
                plt.plot(
                    [test_row['total.sulfur.dioxide'], neighbor_row['total.sulfur.dioxide']],
                    [test_row['citric.acid'], neighbor_row['citric.acid']],
                    'r--',
                    linewidth=0.5,
                )

        plt.xlabel('total.sulfur.dioxide')
        plt.ylabel('citric.acid')
        plt.title('Scatter Plot of total.sulfur.dioxide vs citric.acid with KNN Neighbors')
        plt.legend()
        plt.show()
        print(end='\n\n\n\n')

## Dados Originais 2 Features:
---

In [ ]:
X_test = orig_test_df_2f.copy()
drop_id_column(X_test)
y_test = X_test.pop('class')

# Loop para iterar sobre 4 valores de 'k' -> (1, 3, 5, 7)
for k in range(1, 8, 2):
    knn = split_and_train(orig_train_df_2f, k)
    y_pred = knn.predict(X_test)

    cm = confusion_matrix(y_test, y_pred)

    print(f"Restultados para k = {k}")
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Previsões')
    plt.ylabel('Valores Reais')
    plt.title('Matriz de Confusão KNN')
    plt.show()

    print(end="\n\n\n")

    print(classification_report(y_test, y_pred))

    print("----" * 20, end="\n\n\n")


## Dados Normalizados 2 Features:
---

In [ ]:
X_test = norm_test_df_2f.copy()
drop_id_column(X_test)
y_test = X_test.pop('class')

for k in range(1, 8, 2):
    knn = split_and_train(norm_train_df_2f, k)
    y_pred = knn.predict(X_test)

    cm = confusion_matrix(y_test, y_pred)

    print(f"Restultados para k = {k}")
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Previsões')
    plt.ylabel('Valores Reais')
    plt.title('Matriz de Confusão KNN')
    plt.show()

    print(end="\n\n\n")

    print(classification_report(y_test, y_pred))

    print("----" * 20, end="\n\n\n")


## Dados Originais 11 Features:
---

In [ ]:
X_test = orig_test_df_11f.copy()
drop_id_column(X_test)
y_test = X_test.pop('class')

for k in range(1, 8, 2):
    knn = split_and_train(orig_train_df_11f, k)
    y_pred = knn.predict(X_test)

    cm = confusion_matrix(y_test, y_pred)

    print(f"Restultados para k = {k}")
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Previsões')
    plt.ylabel('Valores Reais')
    plt.title('Matriz de Confusão KNN')
    plt.show()

    print(end="\n\n\n")

    print(classification_report(y_test, y_pred))

    print("----" * 20, end="\n\n\n")

## Dados Normalizados 11 Features:
---

In [ ]:
X_test = norm_test_df_11f.copy()
drop_id_column(X_test)
y_test = X_test.pop('class')

for k in range(1, 8, 2):
    knn = split_and_train(norm_train_df_11f, k)
    y_pred = knn.predict(X_test)

    cm = confusion_matrix(y_test, y_pred)

    print(f"Restultados para k = {k}")
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Previsões')
    plt.ylabel('Valores Reais')
    plt.title('Matriz de Confusão KNN')
    plt.show()

    print(end="\n\n\n")

    print(classification_report(y_test, y_pred))

    print("----" * 20, end="\n\n\n")

# Secção para responder as perguntas do questionário:
---


## A)
Treine modelos usando o algoritmo KNN para o conjunto de dados Dados_Originais_2Features (não normalizados), variando o valor de k (número de vizinhos mais próximos) entre 1, 3, 5, 7. Para cada modelo treinado, avalie seu desempenho nos dados de teste, reportando a acurácia. Repita o mesmo procedimento com os dados Dados_Normalizados_2Features. Compare as acurácias obtidas nos modelos treinados a partir destes dois conjuntos de dados, analisando se a normalização impactou de alguma forma os resultados. Observe se a mudança no valor de k causou algum impacto no desempenho destes modelos (com e sem normalização dos dados) e, em caso positivo, se as variações no desempenho são as mesmas entre os modelos treinados com mesmo k, mas com dados distintos (dados originais e dados normalizados)

In [ ]:
orig_knn_dict = {f"knn_{k}": split_and_train(orig_train_df_2f, k) for k in range(1, 8, 2)}
norm_knn_dict = {f"knn_{k}": split_and_train(norm_train_df_2f, k) for k in range(1, 8, 2)}

In [ ]:
orig_acc_list = [orig_knn_dict[f"knn_{k}"].score(orig_test_df_2f.drop(['ID', 'class'], axis = 1), orig_test_df_2f['class']) for k in range(1, 8, 2)]
norm_acc_list = [norm_knn_dict[f"knn_{k}"].score(norm_test_df_2f.drop(['ID', 'class'], axis = 1), norm_test_df_2f['class']) for k in range(1, 8, 2)]

In [ ]:
# Dados fornecidos
k_neighbors = [1, 3, 5, 7]

# Criar a figura
plt.figure(figsize=(10, 6))

# Criar o gráfico de linha
plt.plot(k_neighbors, orig_acc_list, marker='o', label='Dados originais')  # 'o' adiciona marcadores aos pontos
plt.plot(k_neighbors, norm_acc_list, marker='^', label='Dados normalizados')  # 'o' adiciona marcadores aos pontos

# Adicionar rótulos e título
plt.xlabel('Número de K Vizinhos Mais Próximos')
plt.ylabel('Acurácia do Modelo')
plt.title('Acurácia do Modelo vs. Número de Vizinhos (K)')
plt.legend()

# Personalizar os ticks do eixo x (opcional, mas recomendado)
plt.xticks(k_neighbors)  # Garante que os ticks do eixo x sejam apenas os valores de k

# Exibir o gráfico
plt.grid(True)  # Adiciona grade para melhor visualização (opcional)
plt.show()

## B)
Considerando o modelo treinado com k=5 utilizando dados não normalizados e com 2 atributos, verifique quem são os k vizinhos mais próximos da instância de teste N1 (liste os respectivos IDs). Verifique como estes vizinhos estão dispostos no espaço de entrada em relação à instância de teste N1 e aos eixos x e y. Após tirar suas conclusões, analise se as mesmas se aplicam às instâncias de teste N2, N3 e N4.

In [ ]:
# prompt: Create a plot from norm_train_df_2f that show features 'total.sulfur.dioxide' and 'citric.acid' in axis, then color them based on 'class', then insert the data points from norm_test_df_2f and make them black dots, with their respective ID above

import matplotlib.pyplot as plt
plt.figure(figsize=(20, 11))
sns.scatterplot(x='total.sulfur.dioxide', y='citric.acid', hue='class', data=orig_train_df_2f)
plt.scatter(orig_test_df_2f['total.sulfur.dioxide'], orig_test_df_2f['citric.acid'], color='black', label='Test Data', marker='^')

for index, row in orig_train_df_2f.iterrows():
    plt.annotate(index, (row['total.sulfur.dioxide']-0.003, row['citric.acid']+0.01), size=8)

for index, row in orig_test_df_2f.iterrows():
    plt.annotate(row['ID'], (row['total.sulfur.dioxide'], row['citric.acid']+0.01))

plt.xlabel('total.sulfur.dioxide')
plt.ylabel('citric.acid')
plt.title('Scatter Plot of total.sulfur.dioxide vs citric.acid')
plt.legend()
plt.show()


In [ ]:
create_plots_and_neighbors(orig_train_df_2f, orig_test_df_2f, k_list=[5])

In [ ]:
## Vizinhos mais próximos de N1:

indices = orig_knn_dict['knn_5'].kneighbors(orig_test_df_2f.iloc[[0], :].drop(['ID', 'class'], axis = 1))[1][0].tolist()

orig_train_df_2f.iloc[indices].sort_values(by = 'ID')

## C)
Treine dois modelos usando o algoritmo KNN com k=5 para os datasets Dados_Normalizados_2Features e Dados_Normalizados_11Features. Aplique os modelos treinados nos respectivos dados de teste, verificando os k-vizinhos mais próximos e a classe predita para a instância N4. Faça perturbações no valor do atributo “citric acid” para a instância N4, substituindo o valor original (1.0) por 0.3 e posteriormente por 0.85 (ou seja, gere duas novas instâncias sintéticas com esta alteração). Repita a classificação destas instâncias sintéticas com os dois modelos (isto é, modelo baseado em 2 atributos e em 11 atributos). Compare os resultados, analisando como a alteração de um atributo impactou o cálculo das distâncias euclidianas e a seleção dos k-vizinhos mais próximos em cada caso.

In [ ]:
print("Valor de teste 'N4' com 2 features:")
display(norm_test_df_2f.iloc[[3], :])

print(end='\n\n\n')

print("Valor de teste 'N4' com 11 features:")
display(norm_test_df_11f.iloc[[3], :])

In [ ]:
# Criando os modelos KNN para o dataset com 2 e 11 features, ambos normalizados.
norm_knn_5_2f = split_and_train(norm_train_df_2f, 5)
norm_knn_5_11f = split_and_train(norm_train_df_11f, 5)

In [ ]:
print(f"Classe predita para o valor de teste 'N4' com 2 features")
print(norm_knn_5_2f.predict(norm_test_df_2f.iloc[[3], :].drop(['ID', 'class'], axis = 1)), end='\n\n')

print(f"Classe predita para o valor de teste 'N4' com 11 features")
print(norm_knn_5_11f.predict(norm_test_df_11f.iloc[[3], :].drop(['ID', 'class'], axis = 1)))

In [ ]:
# Criando novas instâncias sintéticas:
norm_2f_original_n4_instance = norm_test_df_2f.iloc[[3], :].drop(['ID', 'class'], axis = 1)
norm_2f_perturbed_n4_instance_1 = norm_2f_original_n4_instance.copy()
norm_2f_perturbed_n4_instance_1['citric.acid'] = 0.3
norm_2f_perturbed_n4_instance_2 = norm_2f_original_n4_instance.copy()
norm_2f_perturbed_n4_instance_2['citric.acid'] = 0.85

norm_11f_original_n4_instance = norm_test_df_11f.iloc[[3], :].drop(['ID', 'class'], axis = 1)
norm_11f_perturbed_n4_instance_1 = norm_11f_original_n4_instance.copy()
norm_11f_perturbed_n4_instance_1['citric.acid'] = 0.3
norm_11f_perturbed_n4_instance_2 = norm_11f_original_n4_instance.copy()
norm_11f_perturbed_n4_instance_2['citric.acid'] = 0.85

In [ ]:
print("Classe predita para o modelo com 2 atributos:", end="\n\n")
print(f"N4 Normalizado (citric.acid = 1.0):\t\t{norm_knn_5_2f.predict(norm_2f_original_n4_instance)[0]}")
print(f"N4 Perturbação 1 (citric.acid = 0.3):\t\t{norm_knn_5_2f.predict(norm_2f_perturbed_n4_instance_1)[0]}")
print(f"N4 Perturbação 2 (citric.acid = 0.85):\t\t{norm_knn_5_2f.predict(norm_2f_perturbed_n4_instance_2)[0]}")

print(end="\n\n")

print("Classe predita para o modelo com 11 atributos:", end="\n\n")
print(f"N4 Normalizado (citric.acid = 1.0):\t\t{norm_knn_5_11f.predict(norm_11f_original_n4_instance)[0]}")
print(f"N4 Perturbação 1 (citric.acid = 0.3):\t\t{norm_knn_5_11f.predict(norm_11f_perturbed_n4_instance_1)[0]}")
print(f"N4 Perturbação 2 (citric.acid = 0.85):\t\t{norm_knn_5_11f.predict(norm_11f_perturbed_n4_instance_2)[0]}")

In [ ]:
## Vizinhos mais próximos de N4:
print("Vizinhos mais próximos para o modelo com 2 atributos:", end="\n\n")
print(f"N4 Original (citric.acid = 1.0):\t\t{norm_train_df_2f.iloc[norm_knn_5_2f.kneighbors(norm_2f_original_n4_instance)[1][0].tolist()].sort_values(by = 'ID')['ID'].to_list()}")
print(f"N4 Perturbação 1 (citric.acid = 0.3):\t\t{norm_train_df_2f.iloc[norm_knn_5_2f.kneighbors(norm_2f_perturbed_n4_instance_1)[1][0].tolist()].sort_values(by = 'ID')['ID'].to_list()}")
print(f"N4 Perturbação 2 (citric.acid = 0.85):\t\t{norm_train_df_2f.iloc[norm_knn_5_2f.kneighbors(norm_2f_perturbed_n4_instance_2)[1][0].tolist()].sort_values(by = 'ID')['ID'].to_list()}")

print(end="\n\n")

print("Vizinhos mais próximos para o modelo com 11 atributos:", end="\n\n")
print(f"N4 Original (citric.acid = 1.0):\t\t{norm_train_df_11f.iloc[norm_knn_5_11f.kneighbors(norm_11f_original_n4_instance)[1][0].tolist()].sort_values(by = 'ID')['ID'].to_list()}")
print(f"N4 Perturbação 1 (citric.acid = 0.3):\t\t{norm_train_df_11f.iloc[norm_knn_5_11f.kneighbors(norm_11f_perturbed_n4_instance_1)[1][0].tolist()].sort_values(by = 'ID')['ID'].to_list()}")
print(f"N4 Perturbação 2 (citric.acid = 0.85):\t\t{norm_train_df_11f.iloc[norm_knn_5_11f.kneighbors(norm_11f_perturbed_n4_instance_2)[1][0].tolist()].sort_values(by = 'ID')['ID'].to_list()}")